# Prepare data

In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import pandas as pd
from multiprocessing import Pool
from sys import getsizeof

ImportError: DLL load failed: 找不到指定的模組。

## Load image

**Long time part here!!!**

In [3]:
#=====Load training data=========

train_data = []
path = "train/*"

#====Get the file name which under the folder===
files = glob.glob(path)
files.sort()

for File in files:
    im = Image.open(File)
    
######Resize the images to a common size######
    im = im.resize((120,120),Image.BILINEAR)
    im = np.asarray(im)
    
#=======Normalize the value of images========
    im = im/255.
    
#=== Transpose the images to channel first, it will be fast at training ===
    train_data.append(np.transpose(im, (2, 0, 1)))
    
train_data = np.asarray(train_data)
print("Training data size:",train_data.shape)

NameError: name 'glob' is not defined

In [3]:
#=====Load validation data=========

valid_data = []
path = "valid/*"

#====Get the file name which under the folder===
files = glob.glob(path)
files.sort()

for File in files:
    im = Image.open(File)
    
######Resize the images to a common size######
    im = im.resize((120,120),Image.BILINEAR)
    im = np.asarray(im)
    
#=======Normalize the value of images========
    im = im/255.
    
#=== Transpose the images to channel first, it will be fast at validation ===
    valid_data.append(np.transpose(im, (2, 0, 1)))
valid_data = np.asarray(valid_data)
print("Valid data size:",valid_data.shape)

('Valid data size:', (2200, 3, 120, 120))


In [4]:
#=====Load testing data=========

test_data = []
path = "test/*"

#====Get the file name which under the folder===
files = glob.glob(path)
files.sort()

for File in files:
    im = Image.open(File)
    
######Resize the images to a common size######
    im = im.resize((120,120),Image.BILINEAR)
    im = np.asarray(im)
    
#=======Normalize the value of images========
    im = im/255.
    
#=== Transpose the images to channel first, it will be fast at training ===
    test_data.append(np.transpose(im, (2, 0, 1)))
    
test_data = np.asarray(test_data)
print("Testing data size:",test_data.shape)

('Testing data size:', (3000, 3, 120, 120))


In [5]:
#====== Print the data size(byte)======
print("train data:",getsizeof(train_data))
print("test  data:",getsizeof(test_data))
print("valid data:",getsizeof(valid_data))

('train data:', 6842880144)
('test  data:', 1036800144)
('valid data:', 760320144)


## Make training label

In [6]:
#=====Make training Labels=============

#===read the csv file===
df = pd.read_csv('train_label.csv')
train_labels = df.values
print(train_labels.shape)

#===make one hot format labels===
for i in range(19800):
    if train_labels[i][1]:
        train_labels[i][0] = 0
    else:
        train_labels[i][0] = 1

(19800, 2)


In [7]:
#=====Make training Label=============

#===read the csv file===
df = pd.read_csv('valid_label.csv')
valid_labels = df.values
print(valid_labels.shape)

#===make one hot format labels===
for i in range(2200):
    if valid_labels[i][1]:
        valid_labels[i][0] = 0
    else:
        valid_labels[i][0] = 1

(2200, 2)


# Build Model

##  Create network

In [2]:

import math
import keras
import numpy as np

from keras import optimizers
from keras import activations
from keras import backend as K
from keras.datasets import fashion_mnist
from keras.datasets import mnist
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Conv2D, Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.layers import Dense, MaxPooling2D, Dropout, Flatten


C:\Users\sxaa\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\sxaa\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tens

In [4]:
input_shape = 19800
batch_size = 128
epochs = 10
num_classes = 2
cls_dict={0:'Cat', 1:'Dog'}

In [5]:
############## Build the network here ##########
def get_model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size = (3,3) , input_shape = (3,120,120),  activation='relu', padding='same' ,data_format="channels_first"))
    model.add(Conv2D(filters=32, kernel_size = (3,3) , activation='relu', padding='same' ,data_format="channels_first"))
    model.add(MaxPooling2D(pool_size=(2, 2),strides = (2,2), data_format="channels_first"))
    
    model.add(Conv2D(filters=64, kernel_size = (3,3) , activation='relu', padding='same' ,data_format="channels_first")) 
    model.add(Conv2D(filters=64, kernel_size = (3,3) , activation='relu', padding='same' ,data_format="channels_first"))     
    model.add(MaxPooling2D(pool_size=(2, 2),strides = (2,2), data_format="channels_first"))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
#   model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

In [7]:
model = get_model()
print(model.summary())

KeyboardInterrupt: 

## Training

Notice!!!

You can set the lower size images in loading stage, if you feel the training speed too slow.

In [7]:
############## Start Training ############
# You need to use validation_data=(valid_data,valid_labels) in model.fit !!!!!! #

# compile our model with crossentropy loss and Adam optimizer
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
# Training Process

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=0, mode='auto')
record = model.fit(train_data, train_labels,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  shuffle=True,
                  callbacks=[earlyStopping], 
                  validation_split=0.1)

NameError: name 'train_data' is not defined

In [1]:
################### Show the training and validation result ###################
### you can use "matplotlib.pyplot" to show the result and paste into report###
plt.plot(record.history['acc'],label='acc')
plt.plot(record.history['val_acc'],label='val_acc')
plt.legend(loc='lower right')
plt.show()

NameError: name 'plt' is not defined

# Print out your result to csv file

In [15]:
############# Here is the code who to print the "model" to csv ##############
############### Notice!!! Your model name should be "model" ################
pred = np.argmax(model.predict(test_data), axis=1).reshape(3000,1)
df = pd.DataFrame(pred, columns = ["labels"])
# print(df)
df.to_csv('0410131_result.csv',index_label = "index")